In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, count

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
path_file = r'C:\Workspace\DataAnalytics\Auxilio_Emergencial\csv_aux_emergencial\202103_AuxilioEmergencial.csv'

In [ ]:
df = spark.read.csv(path_file, encoding='ISO-8859-1', sep='";"', header=True)

In [ ]:
df = df.toDF('ANO_MES', 'UF', 'COD_MUNICIPIO','NOME_MUNICIPIO','NIS_BENEF','CPF_BENEF','NOME_BENEF','NIS_RESP','CPF_RESP','NOME_RESP','ENQUADRAMENTO','PARCELA','OBS','VALOR')

In [ ]:
df2 = df.withColumn('ANO_MES', regexp_replace('ANO_MES', '"', ''))\
    .withColumn('VALOR', regexp_replace('VALOR', '"', ''))\
    .withColumn('VALOR', regexp_replace('VALOR', ',', '.'))\
    .withColumn('VALOR', col('VALOR').cast('float'))

In [ ]:
df2.filter("COD_MUNICIPIO IS NOT NULL").show(2)

In [ ]:
df2.filter("COD_MUNICIPIO IS NOT NULL AND CPF_BENEF IS NULL").show(2)

In [ ]:
df2.filter("COD_MUNICIPIO IS NOT NULL AND CPF_BENEF IS NOT NULL")\
    .groupBy(['ANO_MES','COD_MUNICIPIO'])\
    .agg(count('COD_MUNICIPIO').alias('TOTAL_BENEF')).show()

In [ ]:
df2.filter("COD_MUNICIPIO IS NOT NULL AND CPF_BENEF IS NULL")\
    .groupBy(['ANO_MES','COD_MUNICIPIO'])\
    .agg(count('COD_MUNICIPIO').alias('TOTAL_ANONIMOS')).show()

In [ ]:
df_final = df2.filter("COD_MUNICIPIO IS NOT NULL").groupBy(['ANO_MES','COD_MUNICIPIO','ENQUADRAMENTO','PARCELA']).sum('VALOR')

In [ ]:
df_final_order = df_final.orderBy(['COD_MUNICIPIO','ENQUADRAMENTO','PARCELA'])

In [ ]:
ano_mes = df_final_order.select('ANO_MES').limit(1).collect()

In [ ]:
destino = r'C:\Workspace\DataAnalytics\Auxilio_Emergencial\csv_aux_emergencial'
path_destino = destino+'\\'+ano_mes[0][0]

try:
    os.mkdir(path_destino)
except FileExistsError:
    pass

In [ ]:
file_csv = path_destino+'\\'+'AuxilioEmergencial.csv'
df_final_order.toPandas().to_csv(file_csv)

In [ ]:
carga_municipios = df5.select('COD_MUNICIPIO', 'NOME_MUNICIPIO', 'UF').distinct().filter("COD_MUNICIPIO IS NOT NULL")

In [ ]:
csv_municipio = path_destino+'\\'+'municipios.csv'
carga_municipios.toPandas().to_csv(csv_municipio)

path_download = 'C:\\Users\\antoliverjr\\Downloads'
path_historico = 'C:\\Workspace\\DataAnalystics\\Aux_Emergencial\\historico_zip'
path_files = 'C:\\Workspace\\DataAnalystics\\Aux_Emergencial\\csv_aux_emergencial'
url_download = 'https://www.portaldatransparencia.gov.br/download-de-dados/auxilio-emergencial/'

try: os.mkdir(path_files)
except FileExistsError: pass

try: os.mkdir(path_historico)
except FileExistsError: pass

file_name = '202104_AuxilioEmergencial.zip'
path_file_origin = os.path.join(path_download, file_name)
path_file_job = os.path.join(path_historico, file_name)

print(file_name)
print(path_historico)

for files in os.listdir(path_historico):
    print(files)